## SQL и Pandas

В этом документе представлено сравнение базовых операций библиотеки Pandas со схожими запросами в языке структурированных запросов *SQL*.

Для начала импортируем библиотеку:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.__version__
%matplotlib inline

Импортируем данные из файла *tips.csv* в привычную pandas-структуру *DataFrame*. Будем считать, что у нас есть аналогичная таблица базы данных с тем же именем и структурой для SQL.

In [2]:
tips = pd.read_csv('tips.csv')

###### Отобразим первые и последние 6 элементов таблицы:

In [3]:
tips.head(6)    # First 6 elements

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
5,25.29,4.71,Male,No,Sun,Dinner,4


In [4]:
tips.tail(6)    # Last 6 elements

,total_bill,tip,sex,smoker,day,time,size
238,35.83,4.67,Female,No,Sat,Dinner,3
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2
243,18.78,3.00,Female,No,Thur,Dinner,2


### SELECT

SELECT - оператор запроса в языке SQL, возвращающий набор данных из базы. В SQL выбор осуществляется с помощью списка столбцов через запятую (или \*, если требуется выбрать все столбцы):

```SQL
SELECT total_bill, tip, smoker, size
FROM tips
LIMIT 8;
```

В pandas выбор столбцов производится путем передачи списка имен столбцов в *DataFrame*:

In [5]:
tips[['total_bill', 'tip', 'smoker', 'size']].head(8)    # Select columns from Dataframe

,total_bill,tip,smoker,size
0,16.99,1.01,No,2
1,10.34,1.66,No,3
2,21.01,3.50,No,3
3,23.68,3.31,No,2
4,24.59,3.61,No,4
5,25.29,4.71,No,4
6,8.77,2.00,No,2
7,26.88,3.12,No,4


Вызов DataFrame без списка имен столбцов отобразит всю таблицу (аналогично в SQL - \*).

### WHERE

Фильтрация по критериям в SQL производится с помощью конструкции WHERE. Критерии должны быть описаны в форме предикатов (утверждений).

```SQL
SELECT *
FROM tips
WHERE size = 2
LIMIT 5;
```

Фильтрация в DataFrame производится несколькими способами. Наиболее интуитивный использует логическое индексирование. 

В качестве примера, отобразим все элементы таблицы, удовлетворяющие условию `size == 2`.

In [6]:
tips[tips['size'] == 2].head(4)    # Filter data by size == 2

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
3,23.68,3.31,Male,No,Sun,Dinner,2
6,8.77,2.00,Male,No,Sun,Dinner,2
8,15.04,1.96,Male,No,Sun,Dinner,2


Приведенный выше оператор передает в *DataFrame* набор значений `True` и `False`, представленный в виде фильтра pandas *Series*. 

Первые 4 элемента булевого фильтра `size == 2`:

In [7]:
(tips['size'] == 2).head(4)    # Show first 4 elements of boolean filter

0     True
1    False
2    False
3     True
Name: size, dtype: bool

Еще один пример по признаку `sex == 'Male'`:

In [8]:
tips[tips['sex'] == 'Male'].head(4)    # Filter data by sex == Male

,total_bill,tip,sex,smoker,day,time,size
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
5,25.29,4.71,Male,No,Sun,Dinner,4


Pandas позволяет подсчитать количество истиных и ложных значений в таблице, удовлетворяющих выбранному условию:

In [9]:
is_size = tips['size'] == 2
is_size.value_counts()

True     156
False     88
Name: size, dtype: int64

Тоже самое по другому признаку:

In [10]:
is_male = tips['sex'] == 'Male'
is_male.value_counts()

True     157
False     87
Name: sex, dtype: int64

Отдельный доступ к истинным или ложным значениям (несмотря на то, что значение `True` отображается до `False`, индексация производится наоборот: по нулевому индексу - `False`, по первому - `True`):

In [11]:
is_male.value_counts()[0]    # Number of False values

87

In [12]:
is_male.value_counts()[1]    # Number of True values

157

В pandas *DataFrame* может быть передано несколько логических условий с помощью `|` (ИЛИ) и `&` (И). В SQL для этой цели используются операторы `OR` и `AND`.

```SQL
-- tips of more than 5.00 dollars at Dinner meals
SELECT *
FROM tips
WHERE time = 'Dinner' AND tip > 5.00;
```

In [13]:
# tips of more than $5.00 at Dinner meals
tips[(tips['time'] == 'Dinner') & (tips['tip'] > 5.00)]

,total_bill,tip,sex,smoker,day,time,size
23,39.42,7.58,Male,No,Sat,Dinner,4
44,30.40,5.60,Male,No,Sun,Dinner,4
47,32.40,6.00,Male,No,Sun,Dinner,4
52,34.81,5.20,Female,No,Sun,Dinner,4
59,48.27,6.73,Male,No,Sat,Dinner,4
116,29.93,5.07,Male,No,Sun,Dinner,4
155,29.85,5.14,Female,No,Sun,Dinner,5
170,50.81,10.00,Male,Yes,Sat,Dinner,3
172,7.25,5.15,Male,Yes,Sun,Dinner,2
181,23.33,5.65,Male,Yes,Sun,Dinner,2


Другой пример:

```SQL
-- tips by parties of at least 5 diners OR bill total was more than $45
SELECT *
FROM tips
WHERE size >= 5 OR total_bill > 45;
```

In [14]:
# tips by parties of at least 5 diners OR bill total was more than $45
tips[(tips['size'] >= 5) | (tips['total_bill'] > 45)]

,total_bill,tip,sex,smoker,day,time,size
59,48.27,6.73,Male,No,Sat,Dinner,4
125,29.80,4.20,Female,No,Thur,Lunch,6
141,34.30,6.70,Male,No,Thur,Lunch,6
142,41.19,5.00,Male,No,Thur,Lunch,5
143,27.05,5.00,Female,No,Thur,Lunch,6
155,29.85,5.14,Female,No,Sun,Dinner,5
156,48.17,5.00,Male,No,Sun,Dinner,6
170,50.81,10.00,Male,Yes,Sat,Dinner,3
182,45.35,3.50,Male,Yes,Sun,Dinner,3
185,20.69,5.00,Male,No,Sun,Dinner,5


Проверка на *NULL* производится с помощью методов `notna()` и `isna()`. Создадим массив с NaN-значениями:

In [15]:
fr = pd.DataFrame({
        'col1': ['A', 'B', np.NaN, 'C', 'D'],
        'col2': ['F', np.NaN, 'G', 'H', 'I']
})

fr

,col1,col2
0,A,F
1,B,NaN
2,NaN,G
3,C,H
4,D,I


Предположим, у нас есть таблица с такой же структурой. С помощью следующего SQL-запроса можно отобразить записи, в которых отражены значения *NULL* во второй колонке *col 2*:

```SQL
SELECT *
FROM frame
WHERE col2 IS NULL;
```

In [16]:
fr[fr['col2'].isna()]   # Show NaN from 'col 2'

,col1,col2
1,B,NaN


Получить не-NULL значения из *col 1* можно с помощью метода `notna()`. В SQL это команда:

```SQL
SELECT *
FROM frame
WHERE col1 IS NOT NULL;
```

In [17]:
fr[fr['col1'].notna()]   # Show not-NaN from 'col 1'

,col1,col2
0,A,F
1,B,NaN
3,C,H
4,D,I


### GROUP BY

Оператор группировки в pandas звучит и записывается аналогично - `groupby()`. Метод применяется в задачах, где требуется разбить набор данных на группы, применить некоторую функцию (например, агрегации), а затем объединить группы вместе.

В SQL это операция получения количества элементов в каждой группе по всему набору данных. Например:

```SQL
SELECT sex, count(*)
FROM tips
GROUP BY sex;
/*
Female     87
Male      157
*/
```

In [18]:
tips.groupby('sex').size()    # Number of Male / Female elements.

sex
Female     87
Male      157
dtype: int64

Важно, что для получения количества элементов с помощью `groupby()` используется не `count()` метод, а `size()`. Метод подсчета количества элементов, примененный в такой конструкции, выведет количество элементов по всем столбцам.

In [19]:
tips.groupby('sex').count()

,total_bill,tip,smoker,day,time,size
sex,,,,,,
Female,87,87,87,87,87,87
Male,157,157,157,157,157,157


SQL и Pandas позволяют применять несколько функций к одной таблице. Например, требуется посмотреть среднее значение чаевых в зависимости от дня недели.

```SQL
SELECT day, AVG(tip), COUNT(*)
FROM tips
GROUP BY day;
/*
Fri   2.734737   19
Sat   2.993103   87
Sun   3.255132   76
Thur  2.771452   62
*/
```

Применим метод `agg()`, в котором укажем среднее значение чаевых и количество дней, в которые были оставлены те или иные чаевые.

In [20]:
tips.groupby('day').agg({'tip': np.mean, 'day': np.size})

,tip,day
day,,
Fri,2.734737,19
Sat,2.993103,87
Sun,3.255132,76
Thur,2.771452,62


Группировка по нескольким столбцам выполняется путём передачи списка столбцов в метод `groupby()`.

```SQL
SELECT smoker, day, COUNT(*), AVG(tip)
FROM tips
GROUP BY smoker, day;
/*
smoker day
No     Fri      4  2.812500
       Sat     45  3.102889
       Sun     57  3.167895
       Thur    45  2.673778
Yes    Fri     15  2.714000
       Sat     42  2.875476
       Sun     19  3.516842
       Thur    17  3.030000
*/
```
В следующем примере в метод группировки передается список выбора по полу и принадлежности к курящим. Агрегирующая функция подсчитывает среднее значение чаевых для каждой строки.

In [21]:
tips.groupby(['sex', 'smoker']).agg({'tip': [np.size, np.mean]})

tip          
               size      mean
sex    smoker                
Female No      54.0  2.773519
       Yes     33.0  2.931515
Male   No      97.0  3.113402
       Yes     60.0  3.051167

### JOIN

В pandas *JOIN* выполняется с помощью методов `join()` и `merge()`. У каждого метода есть набор параметров, позволяющих указать тип объединения (слева, справа, ..) или столбцы для объединения (именя столбцов или их индексы)

In [22]:
np.random.seed(1)
df1 = pd.DataFrame({'key': ['A', 'B', 'C', 'D'], 'value': np.random.randn(4)})
df2 = pd.DataFrame({'key': ['B', 'D', 'D', 'E'], 'value': np.random.randn(4)})

### INNER JOIN

**Inner Join** необходим для получения только тех строк, для которых существует соответствие записей главной таблицы и присоединяемой.

```SQL
SELECT *
FROM df1
INNER JOIN df2
  ON df1.key = df2.key;
```

In [23]:
# merge performs an INNER JOIN by default
pd.merge(df1, df2, on='key')

,key,value_x,value_y
0,B,-0.611756,0.865408
1,D,-1.072969,-2.301539
2,D,-1.072969,1.744812


merge() also offers parameters for cases when you’d like to join one DataFrame’s column with another DataFrame’s index.

### LEFT OUTER JOIN

```SQL
-- show all records from df1
SELECT *
FROM df1
LEFT OUTER JOIN df2
  ON df1.key = df2.key;
```

In [24]:
# show all records from df1
pd.merge(df1, df2, on='key', how='left')

,key,value_x,value_y
0,A,1.624345,NaN
1,B,-0.611756,0.865408
2,C,-0.528172,NaN
3,D,-1.072969,-2.301539
4,D,-1.072969,1.744812


### RIGHT JOIN

```SQL
-- show all records from df2
SELECT *
FROM df1
RIGHT OUTER JOIN df2
  ON df1.key = df2.key;
```

In [25]:
# show all records from df2
pd.merge(df1, df2, on='key', how='right')

,key,value_x,value_y
0,B,-0.611756,0.865408
1,D,-1.072969,-2.301539
2,D,-1.072969,1.744812
3,E,NaN,-0.761207


### FULL JOIN

```SQL
-- show all records from both tables
SELECT *
FROM df1
FULL OUTER JOIN df2
  ON df1.key = df2.key;
```

Pandas позволяет сделать полное объединение (с учетом повторения индексов), независимо от того, найдены ли соответствующие столбцы или нет.

In [26]:
# show all records from both frames
pd.merge(df1, df2, on='key', how='outer')

,key,value_x,value_y
0,A,1.624345,NaN
1,B,-0.611756,0.865408
2,C,-0.528172,NaN
3,D,-1.072969,-2.301539
4,D,-1.072969,1.744812
5,E,NaN,-0.761207


### UNION

В pandas оператор `UNION` реализован на базе функции конкатенации - `concat`:

```SQL
SELECT city, rank
FROM df1
UNION ALL
SELECT city, rank
FROM df2;
/*
         city  rank
      Chicago     1
San Francisco     2
New York City     3
      Chicago     1
       Boston     4
  Los Angeles     5
*/
```

In [27]:
df1 = pd.DataFrame({'city': ['Chicago', 'San Francisco', 'New York City'], 
                    'rank': range(1, 4)})
df1

,city,rank
0,Chicago,1
1,San Francisco,2
2,New York City,3


In [28]:
df2 = pd.DataFrame({'city': ['Chicago', 'Boston', 'Los Angeles'],
                    'rank': [1, 4, 5]})
df2

,city,rank
0,Chicago,1
1,Boston,4
2,Los Angeles,5


In [29]:
pd.concat([df1, df2])

,city,rank
0,Chicago,1
1,San Francisco,2
2,New York City,3
0,Chicago,1
1,Boston,4
2,Los Angeles,5


SQL `UNION` аналогичен `UNION ALL`, однако UNION удаляет дублирующиеся строки.

```SQL
SELECT city, rank
FROM df1
UNION
SELECT city, rank
FROM df2;
-- notice that there is only one Chicago record this time
/*
         city  rank
      Chicago     1
San Francisco     2
New York City     3
       Boston     4
  Los Angeles     5
*/
```
В pandas это можно реализовать в связке функций конкатенации - `concat()` и удаления дублей `drop_duplicates()`.

In [30]:
pd.concat([df1, df2]).drop_duplicates()

,city,rank
0,Chicago,1
1,San Francisco,2
2,New York City,3
1,Boston,4
2,Los Angeles,5


## Агрегирующие и аналитические функции

### N строк со смещением

```SQL
SELECT * FROM tips
ORDER BY tip DESC
LIMIT 10 OFFSET 5;
```

In [31]:
tips.nlargest(10 + 5, columns='tip').tail(10)

,total_bill,tip,sex,smoker,day,time,size
183,23.17,6.50,Male,Yes,Sun,Dinner,4
214,28.17,6.50,Female,Yes,Sat,Dinner,3
47,32.40,6.00,Male,No,Sun,Dinner,4
239,29.03,5.92,Male,No,Sat,Dinner,3
88,24.71,5.85,Male,No,Thur,Lunch,2
181,23.33,5.65,Male,Yes,Sun,Dinner,2
44,30.40,5.60,Male,No,Sun,Dinner,4
52,34.81,5.20,Female,No,Sun,Dinner,4
85,34.83,5.17,Female,No,Thur,Lunch,4
211,25.89,5.16,Male,Yes,Sat,Dinner,4


### N строк в группе

```SQL
-- Oracle's ROW_NUMBER() analytic function
SELECT * FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER(PARTITION BY day ORDER BY total_bill DESC) AS rn
  FROM tips t
)
WHERE rn < 4
ORDER BY day, rn;
```

Приведенный ниже пример выведет следующее:  

Отсортированную таблицу по параметру *total_bill* в убывающем порядке.  
Таблица сгруппирована по дням недели - *day*.  
Количество отображаемых элементов *3*.  
Счетчик для каждого дня в таблице - *rn*.  

In [32]:
(tips.assign(rn=tips.sort_values(['total_bill'], ascending=False)
             .groupby(['day'])
             .cumcount() + 1)
 .query('rn < 4')
 .sort_values(['day', 'rn']))


,total_bill,tip,sex,smoker,day,time,size,rn
95,40.17,4.73,Male,Yes,Fri,Dinner,4,1
90,28.97,3.00,Male,Yes,Fri,Dinner,2,2
96,27.28,4.00,Male,Yes,Fri,Dinner,2,3
170,50.81,10.00,Male,Yes,Sat,Dinner,3,1
212,48.33,9.00,Male,No,Sat,Dinner,4,2
59,48.27,6.73,Male,No,Sat,Dinner,4,3
156,48.17,5.00,Male,No,Sun,Dinner,6,1
182,45.35,3.50,Male,Yes,Sun,Dinner,3,2
184,40.55,3.00,Male,Yes,Sun,Dinner,2,3
197,43.11,5.00,Female,Yes,Thur,Lunch,4,1


Тоже самое, используя функцию `rank(method='first')`

In [33]:
(tips.assign(rnk=tips.groupby(['day'])['total_bill']
             .rank(method='first', ascending=False))
 .query('rnk < 3')
 .sort_values(['day', 'rnk']))


,total_bill,tip,sex,smoker,day,time,size,rnk
95,40.17,4.73,Male,Yes,Fri,Dinner,4,1.0
90,28.97,3.00,Male,Yes,Fri,Dinner,2,2.0
170,50.81,10.00,Male,Yes,Sat,Dinner,3,1.0
212,48.33,9.00,Male,No,Sat,Dinner,4,2.0
156,48.17,5.00,Male,No,Sun,Dinner,6,1.0
182,45.35,3.50,Male,Yes,Sun,Dinner,3,2.0
197,43.11,5.00,Female,Yes,Thur,Lunch,4,1.0
142,41.19,5.00,Male,No,Thur,Lunch,5,2.0


```SQL
-- Oracle's RANK() analytic function
SELECT * FROM (
  SELECT
    t.*,
    RANK() OVER(PARTITION BY sex ORDER BY tip) AS rnk
  FROM tips t
  WHERE tip < 2
)
WHERE rnk < 3
ORDER BY sex, rnk;
```

Найдем параметр *tips* с условием **(rank < 3)** для каждого гендерного признака *sex* (для мужчин и женщин), при условии **(tips < 2)**.   

Важно, функция `rank(method='min')` остается такой же для `tip` (как функция Oracle `RANK()`)

In [34]:
(tips[tips['tip'] < 2].assign(rnk_min=tips.groupby(['sex'])['tip']
                              .rank(method='min'))
 .query('rnk_min < 4')
 .sort_values(['sex', 'rnk_min']))

,total_bill,tip,sex,smoker,day,time,size,rnk_min
67,3.07,1.00,Female,Yes,Sat,Dinner,1,1.0
92,5.75,1.00,Female,Yes,Fri,Dinner,2,1.0
111,7.25,1.00,Female,No,Sat,Dinner,1,1.0
236,12.60,1.00,Male,Yes,Sat,Dinner,2,1.0
237,32.83,1.17,Male,Yes,Sat,Dinner,2,2.0
75,10.51,1.25,Male,No,Sat,Dinner,2,3.0
235,10.07,1.25,Male,No,Sat,Dinner,2,3.0


### UPDATE

```SQL
UPDATE tips
SET tip = tip*2
WHERE tip < 2;
```

In [35]:
tips.loc[tips['tip'] < 2, 'tip'] *= 2
tips.head(10)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,2.02,Female,No,Sun,Dinner,2
1,10.34,3.32,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
5,25.29,4.71,Male,No,Sun,Dinner,4
6,8.77,2.00,Male,No,Sun,Dinner,2
7,26.88,3.12,Male,No,Sun,Dinner,4
8,15.04,3.92,Male,No,Sun,Dinner,2
9,14.78,3.23,Male,No,Sun,Dinner,2


### DELETE

```SQL
DELETE FROM tips
WHERE tip > 9;
```

В pandas вместо операции удаления используется функция фильтрации элементов, которые должны остаться в таблице

In [36]:
tips = tips.loc[(tips['tip'] > 2.0) & (tips['tip'] < 2.5)]
tips

,total_bill,tip,sex,smoker,day,time,size
0,16.99,2.02,Female,No,Sun,Dinner,2
22,15.77,2.23,Female,No,Sat,Dinner,2
25,17.81,2.34,Male,No,Sat,Dinner,4
33,20.69,2.45,Female,No,Sat,Dinner,4
38,18.69,2.31,Male,No,Sat,Dinner,3
40,16.04,2.24,Male,No,Sat,Dinner,3
48,28.55,2.05,Male,No,Sun,Dinner,3
66,16.45,2.47,Female,No,Sat,Dinner,2
68,20.23,2.01,Male,No,Sat,Dinner,2
69,15.01,2.09,Male,Yes,Sat,Dinner,2


In [37]:
tips = tips.loc[tips['sex'] == 'Male']
tips

,total_bill,tip,sex,smoker,day,time,size
25,17.81,2.34,Male,No,Sat,Dinner,4
38,18.69,2.31,Male,No,Sat,Dinner,3
40,16.04,2.24,Male,No,Sat,Dinner,3
48,28.55,2.05,Male,No,Sun,Dinner,3
68,20.23,2.01,Male,No,Sat,Dinner,2
69,15.01,2.09,Male,Yes,Sat,Dinner,2
84,15.98,2.03,Male,No,Thur,Lunch,2
120,11.69,2.31,Male,No,Thur,Lunch,2
129,22.82,2.18,Male,No,Thur,Lunch,3
166,20.76,2.24,Male,No,Sun,Dinner,2


In [38]:
tips = tips.loc[tips['smoker'] == 'Yes']
tips

,total_bill,tip,sex,smoker,day,time,size
69,15.01,2.09,Male,Yes,Sat,Dinner,2
193,15.48,2.02,Male,Yes,Thur,Lunch,2
208,24.27,2.03,Male,Yes,Sat,Dinner,2
220,12.16,2.20,Male,Yes,Fri,Lunch,2
237,32.83,2.34,Male,Yes,Sat,Dinner,2


In [39]:
tips = tips.loc[tips['time'] == 'Lunch']
tips

,total_bill,tip,sex,smoker,day,time,size
193,15.48,2.02,Male,Yes,Thur,Lunch,2
220,12.16,2.20,Male,Yes,Fri,Lunch,2
